<a href="https://colab.research.google.com/github/GeeK1224/cau_ml_project/blob/main/cau_ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Project Team ATES(отец)
This is our implementation of `CAU | ML Challenge | Final 2023`

In [1]:
import pandas as pd
import numpy as np

#### Step 1: Data Analysis
After we imported all libraries we needed we can start from analyzing the data given

In [2]:
test_feature = pd.read_csv("kaggle_test_features_public.csv")
train_data = pd.read_csv("kaggle_train_data_public.csv")
sample = pd.read_csv("sample_submission.csv")

In [11]:
test_feature.columns.values

array(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract',
       'Paperless Billing', 'Payment Method', 'Monthly Charges'],
      dtype=object)

In [ ]:
columns = train_data.columns.values
for column in columns:
  print(f"{column} - {train_data[column].unique()}")

# Data Classification
Here we classified columns according to their usability and overall importance based only on common sense.
Clases:
1. 🟥 unusable
2. 🟩 usable
3. 🟪 questionable

| Column name | Status | Reason |
| --- | --- | --- |
| CustomerID | 🟩 | Just to distinguish users |
| Count | 🟥 | Only one value equal to 1 |
| Country | 🟥 | U.S.A. only |
| State | 🟥 | California only |
| City | 🟪 | Questionable, there are California cities, maybe there is some correlation based on city (can be used as categorical data) |
| Zip Code | 🟥 | a useless value that will throw off the weights |
| Lat Long | 🟥 | a useless value that will throw off the weights |
| Latitude | 🟥 | a useless value that will throw off the weights |
| Longitude | 🟥 | a useless value that will throw off the weights |
| Gender | 🟩 | Categorical data, contains useful information |
| Senior Citizen | 🟩 | Categorical data, contains useful information |
| Partner | 🟩 | Categorical data, contains useful information |
| Dependents | 🟩 | Categorical data, contains useful information |
| Tenure Months | 🟩 | Contains useful information, very important |
| Phone Service | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Multiple Lines | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Internet Service | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Online Security | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Online Backup | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Device Protection | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Tech Support | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Streaming TV | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Streaming Movies | 🟥 | In general, it’s difficult to say anything based on the lower values. |
| Contract | 🟩 | Categorical data, contains useful information, very useful |
| Paperless Billing | 🟥 | Not useful because the value can be obtained from another more informative column |
| Payment Method | 🟪 | questionable |
| Monthly Charges | 🟪 | questionable |
| Churn Score | 🟪 |  questionable |
| Churn Value | 🟩 |  Contain useful data |
| CLTV | 🟪 |  quetionable |
| Churn Reason | 🟪 | Well, I don’t know, when canceling a subscription or deleting a program, companies often ask you to leave a feedback, maybe you can classify problems under general cases and push the values |

In [3]:
test_feature["State"].unique()

array(['California'], dtype=object)

In [4]:
test_feature.shape

(986, 27)

In [5]:
train_data["Gender"].describe()

count     3944
unique       2
top       Male
freq      1973
Name: Gender, dtype: object

In [14]:
train_data.describe()

,Count,Zip Code,Latitude,Longitude,Tenure Months,Monthly Charges,Churn Score,Churn Value,CLTV
count,3944.0,3944.000000,3944.000000,3944.000000,3944.000000,3944.000000,3944.000000,3944.000000,3944.000000
mean,1.0,93504.840517,36.282735,-119.811722,32.413540,64.631415,58.683824,0.264959,4392.830122
std,0.0,1868.032274,2.460654,2.140234,24.556004,29.991988,21.497163,0.441368,1181.398114
min,1.0,90001.000000,32.555828,-124.301372,0.000000,18.400000,5.000000,0.000000,2003.000000
25%,1.0,92077.250000,34.037319,-121.806872,9.000000,35.637500,41.000000,0.000000,3468.750000
50%,1.0,93550.500000,36.375590,-119.687964,29.000000,70.250000,61.000000,0.000000,4505.000000
75%,1.0,95337.250000,38.158676,-118.057091,55.000000,89.750000,75.000000,1.000000,5374.750000
max,1.0,96161.000000,41.962127,-114.192901,72.000000,118.650000,100.000000,1.000000,6500.000000
